<font size=7 color ="#5499C7" >Audio MNIST</font>
- <font size=5>__Done By__ : Kasra Noorbakhsh</font>
- <font size=5>__UID__  : 810100230</font>
- <font size=5>__Goal__  : Digit and Speaker Recognition</font>
- <font size=5>__Dataset Path__  : ./data</font>

<p align="justify"><font size="6" color="#A9CCE3"><b>Section 0: Setup & imports</b></font><br></p>

In [3]:
import os
import json
import random
import librosa
import numpy as np
import pandas as pd
from glob import glob
import librosa.display
import soundfile as sf
import IPython.display as ipd
import matplotlib.pyplot as plt
from collections import Counter, defaultdict

# reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Paths
DATA_ROOT = "data"

# Check GPU (if using torch later)
try:
    import torch
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Torch device:", device)
except Exception:
    device = None
    print("Torch not available in this environment")

Torch device: cpu


#### <p align="justify"><font size="6" color="#A9CCE3"><b>Section 1: Gather file list & quick metadata</b></font></p>

In [5]:
rows = []
for speaker_folder in sorted(glob(os.path.join(DATA_ROOT, "*"))):
    speaker_id = os.path.basename(speaker_folder)
    if not speaker_id.isdigit(): 
        continue
    for wav_path in glob(os.path.join(speaker_folder, "*.wav")):
        fname = os.path.basename(wav_path)
        # Expect format: digit_speaker_iter.wav e.g. 0_04_20.wav
        parts = fname.replace(".wav","").split("_")
        if len(parts) < 3:
            # log weird filenames
            rows.append({"path": wav_path, "speaker": speaker_id, "digit": None, "iter": None, "flag_bad_name": True})
            continue
        digit = int(parts[0])
        speaker_code = parts[1]
        iteration = parts[2]
        rows.append({"path": wav_path, "speaker": speaker_id, "digit": digit, "iter": iteration, "flag_bad_name": False})

df = pd.DataFrame(rows)
print("Total files:", len(df))
print(df.head())

Total files: 30000
                  path speaker  digit iter  flag_bad_name
0   data\01\0_01_0.wav      01      0    0          False
1   data\01\0_01_1.wav      01      0    1          False
2  data\01\0_01_10.wav      01      0   10          False
3  data\01\0_01_11.wav      01      0   11          False
4  data\01\0_01_12.wav      01      0   12          False


#### <p align="justify"><font size="6" color="#A9CCE3"><b>Section 2: Durations and sample rate checks</b></font></p>

### <p align="justify"><font size="6" color="#d62b5b">1. 2D Gaussian Distribution</font></p>
<p align="justify">

<p align="justify">

</p>